# NYC Taxi Data Preprocessing
## Imports

In [1]:
!pip install pyshp
!pip install haversine
!pip install geopandas
!pip install pygeos

You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!sudo apt install libspatialindex-dev

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install Rtree

You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install pyshp

You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install haversine

You should consider upgrading via the 'C:\Users\julia\PycharmProjects\HumanMobilityPredictionMA\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Use SQL-Alchemy for queries to data.

In [6]:
# Imports
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
from tqdm import tqdm
import time
from datetime import datetime
import timeit
import gc

# libraries to visualize data
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# Access to a database to operate on big data
import sqlalchemy
from sqlalchemy import create_engine
nyc_database = create_engine('sqlite:///nyc_database.db')

# compute additional features
from haversine import haversine
from pandas import DataFrame
from numba import njit
from math import radians, cos, sin, asin, sqrt

ModuleNotFoundError: No module named 'descartes'

Import to SQL database for the relevant columns into Alchemy.

In [ ]:
# Insert datasets to the db
j, chunksize = 1, 100000
for month in range(1,2):
    fp = "./trip_data/trip_data_{0}.csv".format(month)
    for df in pd.read_csv(fp, chunksize=chunksize, iterator=True):
        df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns})
        df.drop(['vendor_id', 'rate_code', 'store_and_fwd_flag', 'trip_time_in_secs', 'trip_distance'],
						 axis=1, inplace=True)
        df.index += j
        df.to_sql('trip_record', nyc_database, if_exists='append')
        j = df.index[-1] + 1
del df

Show first 100 records for the database

In [ ]:
# Check what has been inserted
pd.read_sql_query('SELECT * FROM trip_record LIMIT 100', nyc_database)

Helper values to verify data integrity and validity.

In [ ]:
# Helper values to define validity

# Find the NYC outliers
min_lat = 40.5774
max_lat = 40.9176
min_long = -74.15
max_long = -73.7004

# valid area (bounding box around the 4 states)
min_lat = 38.8472
max_lat = 45.0153
min_long = -80.5243
max_long = -71.7517


# upper and lower limits for the datetimes
upper = '2013-12-31 23:59:59'
lower = '2013-01-01 00:00:00'

# Logic operators to connect where clauses
OR = ' OR '
AND = ' AND '
NOT = 'NOT '

Build queries to check for incorrect rows, outliers (boundary box NYC), malformed data (timestamps, NULL-values, etc.)

In [ ]:
# Records with not null coordinates
c1 = ( '((pickup_latitude IS NOT NULL AND dropoff_latitude IS NOT NULL) AND ' + 
         '(pickup_longitude IS NOT NULL AND dropoff_longitude IS NOT NULL))')

# Records with coordinates in the range
c2 = ('((dropoff_longitude BETWEEN :min_long AND :max_long ) AND '+
         '(dropoff_latitude BETWEEN :min_lat AND :max_lat) AND '+
      '(pickup_longitude BETWEEN :min_long AND :max_long ) AND '+
         '(pickup_latitude BETWEEN :min_lat AND :max_lat))')

# Records with pickup coordinates not equal to dropoff
c3 = ('((pickup_latitude != dropoff_latitude) OR ' +
         '(pickup_longitude != dropoff_longitude ))')

# Records with different coordinates but distance greater 0
c4 = '(trip_distance > 0.1)'

# Records with pickup time smaller than dropoff time
c5 = '(pickup_datetime < dropoff_datetime)'

# Records with dates in the right range
c6 = ('((pickup_datetime <= :upper AND pickup_datetime >= :lower ) AND (dropoff_datetime <= :upper AND dropoff_datetime >= :lower ))')

# Not Zero passengers and passenger count in the range
c7 = '((passenger_count BETWEEN 1 AND 6) AND passenger_count IS NOT null )'

# Records with not null datetimes
c8 = '(pickup_datetime IS NOT NULL AND dropoff_datetime IS NOT NULL)'

where_clause = '(' + c1 + AND + c2 + AND + c3 + AND + c5 + AND + c6 + AND + c7 + AND + c8 + ')'

# Count valid rows
query = ('SELECT count(*) as number FROM trip_record WHERE ' + where_clause)
res = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})

# Save the result 
valid = res['number'][0]
print('Number of valid rows: ', valid)


In [ ]:
# Select the valid records
query = ('SELECT *  FROM trip_record WHERE '+ where_clause )
valid_records = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})

# Shape of the result
valid_records.shape

Helper function to compute haversine distance (computed distance between two locations), trip time and time differences as well as speed.

In [ ]:
from haversine import haversine

# Compute the haversine distance of a single trip (var_row) in Km
def haversine_distance(var_row):
  h_distance =  haversine((var_row["pickup_latitude"], var_row["pickup_longitude"]), 
                     (var_row["dropoff_latitude"], var_row["dropoff_longitude"]))
	# Round the computed distance to the third decimal
  return round(h_distance, 3)

In [ ]:
from datetime import datetime

# Time difference within a trip
def compute_trip_time(var_row):
  elapsedTime = datetime.strptime(var_row["dropoff_datetime"], '%Y-%m-%d %H:%M:%S') - datetime.strptime(var_row["pickup_datetime"], '%Y-%m-%d %H:%M:%S')
  duration_seconds = elapsedTime.total_seconds()
  duration_hours = duration_seconds/3600
	# Round the computed hours to the third decimal
  return round(duration_hours, 3)

# Time difference between two different trips
def compute_time_difference_trips(datetime_1, datetime_2):
  elapsedTime = datetime.strptime(datetime_2, '%Y-%m-%d %H:%M:%S') - datetime.strptime(datetime_1, '%Y-%m-%d %H:%M:%S')
  duration_seconds = elapsedTime.total_seconds()
  duration_hours = duration_seconds/3600
	# Round the computed hours to the third decimal
  return round(duration_hours, 3)

In [ ]:
# Estimate the average estimated speed of the trip in km/h
def estimate_speed(var_row):
  speed = var_row["haversine_distance"]/var_row["trip_time"]
	# Round the estimated speed to the first decimal
  return round(speed, 1)

In [ ]:
# Convert datetimes type
valid_records["pickup_datetime"] = pd.to_datetime(valid_records["pickup_datetime"])
valid_records["dropoff_datetime"] = pd.to_datetime(valid_records["dropoff_datetime"])

In [ ]:
# Compute time difference in seconds
valid_records['trip_time'] = (valid_records['dropoff_datetime'] - valid_records['pickup_datetime']).dt.total_seconds()

In [ ]:
from numba import njit

# fix problematic times and convert time from seconds into hours
@njit
def refine_time(time_difference):
  res = np.empty(time_difference.shape)
	# Cycle through all the trip_times
  for i in range(len(time_difference)):
		# Convert from seconds to hours
    duration_hours = time_difference[i]/3600.0
		# If time is to small, set it to the minimum
    if duration_hours < 0.005:
      trip_time = 0.005
    else:
      trip_time = round(duration_hours, 3)
    res[i] = trip_time
  return res

In [ ]:
# Convert the trip_times
valid_records['trip_time'] = refine_time(valid_records['trip_time'].values)

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine_np( lat1, lon1, lat2, lon2):
    # earth radius
    R = 6371.0
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = R * c
    return np.round(km, 3)

In [ ]:
# Compute haversine_distance
valid_records['haversine_distance'] = haversine_np(valid_records["pickup_latitude"], valid_records["pickup_longitude"], 
                     valid_records["dropoff_latitude"], valid_records["dropoff_longitude"])

In [ ]:
# Compute the estimated speed
valid_records['speed'] = round(valid_records['haversine_distance']/valid_records['trip_time'], 1)

In [ ]:
# Tresholds
haversine_distance_t = (0.1, 70)
trip_time_t = (0.1, 4.0)
speed_t = (5, 60)

# Define more restrictive thresholds for the intermediate trips
intermediate_distance_t = (0.5, 20)
intermediate_trip_time_t = (0.1, 1.0)
intermediate_speed_t = (5, 40)

Remove outliers in terms of speed (must be incorrect data points).

In [ ]:
# Remove the records
valid_records = valid_records.loc[(valid_records.speed >= speed_t[0]) &
(valid_records.speed <= speed_t[1])]

# Plot the new curve
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["speed"])), np.sort(valid_records["speed"].values), color = 'green')
plt.xlabel('index')
plt.ylabel('speed in km/h')
plt.show()

In [ ]:
# Remove the records
d_records = valid_records.loc[(valid_records.haversine_distance >= haversine_distance_t[0]) &
(valid_records.haversine_distance <= haversine_distance_t[1])]

# Plot the new curve
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["haversine_distance"])), np.sort(valid_records["haversine_distance"]), color = 'blue')
plt.xlabel('index')
plt.ylabel('haversine_distance in km')
plt.show()

In [ ]:
# Remove the records
valid_records = valid_records.loc[(valid_records.trip_time >= trip_time_t[0]) &
(valid_records.trip_time <= trip_time_t[1])]

# Plot the new curve
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["trip_time"])), np.sort(valid_records["trip_time"]), color = 'red')
plt.xlabel('index')
plt.ylabel('trip_time in km')
plt.show()

Generate the trips for our dataset.

In [ ]:
# Retrieve the valid trips
query = ('SELECT medallion, hack_license, pickup_datetime, dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude '+
         'FROM trip_record WHERE '+ where_clause + ' ORDER BY medallion, pickup_datetime')
t1 = datetime.now()
valid_records = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})
t2 = datetime.now()
delta = t2 - t1
print("time for query: {}".format(delta.seconds))

In [ ]:
valid_records["pickup_datetime"] = pd.to_datetime(valid_records["pickup_datetime"])
valid_records["dropoff_datetime"] = pd.to_datetime(valid_records["dropoff_datetime"])

Rename columns and concatenate in order to have aligned rows for the trips.

In [ ]:
# Rename the columns for the dataframe copy
print("Renaming the new columns...")
new_columns_name = np.apply_along_axis(lambda column: column+'_b', 0, np.array(valid_records.columns))

# Copy the dataframe and use the new columns names (takes some time..)
print("Copying the dataframe...")
valid_records_copy  = valid_records.copy()
valid_records_copy.columns = new_columns_name

# Shift the copy dataframe
print("Shifting the copied dataframe...")
valid_records_copy = valid_records_copy.shift(-1)

# concatenate the dataframes 
# NB we have used dropna() because the last row will have no a next trip
print("Concatenating the two dataframes...")
concatenated_dataframes = pd.concat([valid_records, valid_records_copy], axis=1, sort=False).dropna()

In [ ]:
# check for columns types
concatenated_dataframes.dtypes

In [ ]:
# Compute the features of the intermediate trip

# intermediate time
concatenated_dataframes['intermediate_time'] = (concatenated_dataframes['pickup_datetime_b'] - 
                                                concatenated_dataframes['dropoff_datetime']).dt.total_seconds()
concatenated_dataframes['intermediate_time'] = refine_time(concatenated_dataframes['intermediate_time'].values)

# intermediate distance
concatenated_dataframes['intermediate_distance'] = haversine_np(concatenated_dataframes["pickup_latitude_b"].values, concatenated_dataframes["pickup_longitude_b"].values, 
                     concatenated_dataframes["dropoff_latitude"].values, concatenated_dataframes["dropoff_longitude"].values)

# intermediate speed
concatenated_dataframes['intermediate_speed'] = round(concatenated_dataframes['intermediate_distance']/concatenated_dataframes['intermediate_time'], 1)

In [ ]:
# Min time allowed between two trips and min intermediate trip length
min_time = intermediate_trip_time_t[0]

# Max time allowed between two trips and max intermediate trip length
max_time = intermediate_trip_time_t[1]

# Min distance allowed between two trips and min intermediate trip distance
min_distance = intermediate_distance_t[0]

# Max distance allowed between two trips and max intermediate trip distance
max_distance = intermediate_distance_t[1]

# Min speed allowed between two trips and min intermediate trip speed
min_speed = intermediate_speed_t[0]

# Max speed allowed between two trips and max intermediate trip speed
max_speed = intermediate_speed_t[1]

CCheck for all the possible trips that lie within the defined ranges for time, distance and speed.

In [ ]:
# Filter the possible trips using the ranges
candidates = concatenated_dataframes.loc[((concatenated_dataframes['medallion'] == concatenated_dataframes['medallion_b']) & (concatenated_dataframes['hack_license'] == concatenated_dataframes['hack_license_b'])) &
                                         ((concatenated_dataframes['intermediate_time'] <= max_time) & (concatenated_dataframes['intermediate_time'] >= min_time)) & 
                                         ((concatenated_dataframes['intermediate_distance'] <= max_distance) & (concatenated_dataframes['intermediate_distance'] >= min_distance)) &
																				 ((concatenated_dataframes['intermediate_speed'] <= max_speed) & (concatenated_dataframes['intermediate_speed'] >= min_speed))]

candidates.shape

In [ ]:
# Select just the desired columns
intermediate_trips = candidates[['medallion', 'hack_license', 'dropoff_datetime', 'pickup_datetime_b', 'dropoff_longitude', 'dropoff_latitude',
                                 'pickup_longitude_b','pickup_latitude_b', 'intermediate_distance', 'intermediate_time','intermediate_speed']]

# Rename the columns
intermediate_trips = intermediate_trips.rename(columns={'medallion':'medallion', 'hack_license':'hack_license', 'dropoff_datetime':'pickup_datetime', 'pickup_datetime_b':'dropoff_datetime', 
                                   'pickup_longitude_b':'dropoff_longitude',
                                   'pickup_latitude_b':'dropoff_latitude', 
                                    'dropoff_longitude':'pickup_longitude',
                                   'dropoff_latitude':'pickup_latitude', 
                                   'intermediate_distance': 'haversine_distance',
                                   'intermediate_time':'trip_time', 'intermediate_speed':'speed'})

In [ ]:
# Select just the desired columns
intermediate_trips = candidates[['medallion', 'hack_license', 'dropoff_datetime', 'pickup_datetime_b', 'dropoff_longitude', 'dropoff_latitude',
                                 'pickup_longitude_b','pickup_latitude_b', 'intermediate_distance', 'intermediate_time','intermediate_speed']]

# Rename the columns
intermediate_trips = intermediate_trips.rename(columns={'medallion':'medallion', 'hack_license':'hack_license', 'dropoff_datetime':'pickup_datetime', 'pickup_datetime_b':'dropoff_datetime', 
                                   'pickup_longitude_b':'dropoff_longitude',
                                   'pickup_latitude_b':'dropoff_latitude', 
                                    'dropoff_longitude':'pickup_longitude',
                                   'dropoff_latitude':'pickup_latitude', 
                                   'intermediate_distance': 'haversine_distance',
                                   'intermediate_time':'trip_time', 'intermediate_speed':'speed'})

Generate all the valid records from the trip data and valid records data.

In [ ]:
# Union of the original valid trips and the new intermediate trips
all_valid_records = pd.concat([valid_records, intermediate_trips], axis=0)

# Sort the records by medallion and pickup_time
all_valid_records = all_valid_records.sort_values(by=['medallion', 'pickup_datetime'])

all_valid_records.shape

In [ ]:
# speed
all_valid_records = all_valid_records.loc[(all_valid_records.speed >= speed_t[0]) & 
                                          (all_valid_records.speed <= speed_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["speed"])), np.sort(all_valid_records["speed"].values), color = 'green')
plt.xlabel('index')
plt.ylabel('speed in km/h')
plt.show()

In [ ]:
# haversine_distance
all_valid_records = all_valid_records.loc[(all_valid_records.haversine_distance >= haversine_distance_t[0]) & 
                                          (all_valid_records.haversine_distance <= haversine_distance_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["haversine_distance"])), np.sort(all_valid_records["haversine_distance"]), color = 'blue')
plt.xlabel('index')
plt.ylabel('haversine_distance in km')
plt.show()

In [ ]:
# trip_time
all_valid_records = all_valid_records.loc[(all_valid_records.trip_time >= trip_time_t[0]) & 
                                          (all_valid_records.trip_time <= trip_time_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["trip_time"])), np.sort(all_valid_records["trip_time"]), color = 'red')
plt.xlabel('index')
plt.ylabel('trip_time in km')
plt.show()

In [ ]:
# pickup
max_pickup_date = all_valid_records["pickup_datetime"].max()
min_pickup_date= all_valid_records["pickup_datetime"].min()


# dropoff
max_dropoff_date = all_valid_records["dropoff_datetime"].max()
min_dropoff_date= all_valid_records["dropoff_datetime"].min()

print("Pickup: ")
print("Min Date: ", min_pickup_date)
print("Max Date: ", max_pickup_date)
print(" ")
print("Dropoff: ")
print("Min Date: ", min_dropoff_date)
print("Max Date: ", max_dropoff_date)

In [ ]:
# pickup
max_pickup_longitude = all_valid_records["pickup_longitude"].max()
min_pickup_longitude= all_valid_records["pickup_longitude"].min()
max_pickup_latitude = all_valid_records["pickup_latitude"].max()
min_pickup_latitude= all_valid_records["pickup_latitude"].min()

# dropoff
max_dropoff_latitude = all_valid_records["dropoff_latitude"].max()
min_dropoff_latitude= all_valid_records["dropoff_latitude"].min()
max_dropoff_longitude = all_valid_records["dropoff_longitude"].max()
min_dropoff_longitude= all_valid_records["dropoff_longitude"].min()

print("Pickup: ")
print("Min Latitude: ", min_pickup_latitude)
print("Min Longitude: ", min_pickup_longitude)
print("Max Latitude: ", max_pickup_latitude)
print("Max Longitude: ", max_pickup_longitude)
print(" ")
print("Dropoff: ")
print("Min Latitude: ", min_dropoff_latitude)
print("Min Longitude: ", min_dropoff_longitude)
print("Max Latitude: ", max_dropoff_latitude)
print("Max Longitude: ", max_dropoff_longitude)

Medallion (Unique Taxi ID) value counts are displayed.

In [ ]:
# Medallions
all_valid_records['medallion'].value_counts()

The most used Taxis have over 500 trips.

Display mean statistics on haversine distance.

In [ ]:
# haversine_distance
mean_d = all_valid_records["haversine_distance"].mean()
max_d = all_valid_records["haversine_distance"].max()
min_d = all_valid_records["haversine_distance"].min()

print("The mean distance is: ", mean_d)
print("The max distance is: ", max_d)
print("The min distance is: ", min_d)
print("The distances are represented in Km unit")

In [ ]:
# trip_time
mean_t = all_valid_records["trip_time"].mean()
max_t = all_valid_records["trip_time"].max()
min_t = all_valid_records["trip_time"].min()

print("The mean trip_time is: ", mean_t)
print("The max trip_time is: ", max_t)
print("The min trip_time is: ", min_t)
print("The trip_times are represented in hours")

Same for speed:

In [ ]:
# speed
mean_speed = all_valid_records["speed"].mean()
max_speed = all_valid_records["speed"].max()
min_speed = all_valid_records["speed"].min()

print("The mean speed is: ", mean_speed)
print("The max speed is: ", max_speed)
print("The min speed is: ", min_speed)

Generate new features for the time components.

In [ ]:
# Extract the new features
all_valid_records["pickup_week_day"] = all_valid_records.pickup_datetime.dt.dayofweek
all_valid_records["pickup_hour"] = all_valid_records.pickup_datetime.dt.hour
all_valid_records["pickup_month"] = all_valid_records.pickup_datetime.dt.month

In [ ]:
# Extract the required columns
# we use .copy() to unlink this dataframe from the original
locations = all_valid_records[["pickup_longitude", "pickup_latitude","dropoff_longitude", "dropoff_latitude"]].copy()

(Optional) Remove records outside the bounding box (NYC)

In [ ]:
# remove records outside nyc
locations_nyc = locations.loc[(locations.pickup_latitude >= min_lat) &
                                          (locations.pickup_latitude <= max_lat )]
locations_nyc = locations_nyc.loc[(locations_nyc.pickup_longitude >= min_long ) &
                                          (locations_nyc.pickup_longitude <= max_long )]

# remove records outside nyc
locations_nyc = locations_nyc.loc[(locations_nyc.dropoff_latitude >= min_lat) &
                                          (locations_nyc.dropoff_latitude <= max_lat )]
locations_nyc = locations_nyc.loc[(locations_nyc.dropoff_longitude >= min_long ) &
                                          (locations_nyc.dropoff_longitude <= max_long )]

GeoPandas Library to map lat/lon to areas.

In [ ]:
# GeoPandas library
import geopandas as gpd
# As Pandas, also Gpd has its kind of Series
from geopandas import GeoSeries
# We need shapely to import and define objects of Spatial types
from shapely.geometry import Point, LineString, Polygon, asMultiPoint
# I will introduce crs later
import fiona.crs

Import shape file for custom generated zones (.shp)

In [ ]:
# Path to the shape file
fp = "./zones/geo_export_be42d9eb-5829-406d-a057-01bd027a191b.shp"

# Read the data
zones = gpd.read_file(fp)
zones.head()

Convert lon/lat stack to (Geo-)Points

In [ ]:
# Stack latitude and longitude together into a Numpy array of N x 2 elements
pickup = np.vstack((locations_nyc["pickup_longitude"].values, locations_nyc["pickup_latitude"].values)).T 

# Convert each pair of coordinates into a Point object
t1 = datetime.now()
points = [Point(i) for i in pickup]
t2 = datetime.now()
delta = t2 - t1
print("time for conversion: {}".format(delta.seconds))

Generate a GeoSeries from the Points and use Fiona and GeoPandas to generate a GeoDataFrame

In [ ]:
# Create a GeoSeries from the set of Points
gs = GeoSeries(points)

# Use Fiona to set the correct crs to the GeoSeries
gs.crs = fiona.crs.from_epsg(4326)

# Create a GeoDataFrame from the GeoSeries
gd = gpd.GeoDataFrame(geometry=gs)

# Drop any possible Nan or None records
gd =  gd.loc[gd.is_valid]
gd.head()

In [ ]:
# Add pickup coordinates to the geodataframe
gd['pickup_longitude'] = locations_nyc["pickup_longitude"].values
gd['pickup_latitude'] = locations_nyc["pickup_latitude"].values
gd.head()

In [ ]:
#check
gd.crs

Spatial join the zones on the geo coordinates.

In [ ]:
# Perform the spatial join
join = gpd.sjoin(gd, zones, how="inner", op="within")
join.head()

In [ ]:
# pickup_longitude | pickup_latitude | borough | location_i | zone
pickup_non_spacial_data = pd.DataFrame({'pickup_longitude': join['pickup_longitude'], 'pickup_latitude':join['pickup_latitude'] ,
                                 'pickup_borough': join['borough'], 'pickup_location_id': join['location_i'], 'pickup_zone': join['zone']})
pickup_non_spacial_data.head()

Same procedure for the dropoff coordinates:

In [ ]:
# stack latitude and longitude together 
dropoff = np.vstack((locations_nyc["dropoff_longitude"].values, locations_nyc["dropoff_latitude"].values)).T 

# Convert each couple into a point object
t1 = datetime.now()
points = [Point(i) for i in dropoff]
t2 = datetime.now()
delta = t2 - t1
print("time for conversion: {}".format(delta.seconds))

In [ ]:
# Create a GeoSeries from the set of Points
gs = GeoSeries(points)

# Use Fiona to set the correct crs to the GeoSeries
gs.crs = fiona.crs.from_epsg(4326)

# Create a GeoDataFrame from the GeoSeries
gd = gpd.GeoDataFrame(geometry=gs)

# Drop any possible Nan or None records
gd =  gd.loc[gd.is_valid]
gd.head()

In [ ]:
# Add pickup coordinates to the geodataframe
gd['dropoff_longitude'] = locations_nyc["dropoff_longitude"].values
gd['dropoff_latitude'] = locations_nyc["dropoff_latitude"].values

In [ ]:
# Perform the spatial Join
join = gpd.sjoin(gd, zones, how="inner", op="within")
join.head()

In [ ]:
# dropoff_longitude | dropoff_latitude | borough | location_i | zone
dropoff_non_spacial_data = pd.DataFrame({'dropoff_longitude': join['dropoff_longitude'], 'dropoff_latitude':join['dropoff_latitude'] ,
                                 'dropoff_borough': join['borough'], 'dropoff_location_id': join['location_i'], 'dropoff_zone': join['zone']})

In [ ]:
# Check in the full dataframe
all_valid_records[all_valid_records.index.duplicated()]

In [ ]:
# Check in the pickup dataframe
pickup_non_spacial_data[pickup_non_spacial_data.index.duplicated()]

In [ ]:
# Drop duplicates
dropoff_non_spacial_data = dropoff_non_spacial_data.loc[~dropoff_non_spacial_data.index.duplicated(keep='first')]

Generate pickup and dropoff zones from the spacial data

In [ ]:
# pickup zones
pickup_zones = pickup_non_spacial_data[ ['pickup_longitude', 'pickup_latitude', 'pickup_location_id']].copy()

# dropoff zones
dropoff_zones = dropoff_non_spacial_data[ ['dropoff_longitude', 'dropoff_latitude', 'dropoff_location_id']].copy()

# And to be sure let's save them into a csv
PATH = './zones_results'
pickup_zones.to_csv(PATH + '/pickup_zones_1.csv', index=False)
dropoff_zones.to_csv(PATH + '/dropoff_zones_1.csv', index=False)

Generate the trips with zones dataframe

In [ ]:
# join the pickup zones
trips_with_zones = pd.merge(all_valid_records, pickup_zones,  how='left',
                  left_on=['pickup_longitude','pickup_latitude'],
                  right_on = ['pickup_longitude','pickup_latitude'])
trips_with_zones.shape

Eliminate duplicates and join the pickup zones

In [ ]:
# drop duplicates
pickup_zones = pickup_zones.drop_duplicates(subset=['pickup_longitude','pickup_latitude'])

In [ ]:
# join the pickup zones again
trips_with_zones = pd.merge(all_valid_records, pickup_zones,  how='left',
                  left_on=['pickup_longitude','pickup_latitude'],
                  right_on = ['pickup_longitude','pickup_latitude'])

In [ ]:
# drop duplicates
dropoff_zones = dropoff_zones.drop_duplicates(subset=['dropoff_longitude','dropoff_latitude'])

# Join
trips_with_zones = pd.merge(trips_with_zones, dropoff_zones,  how='left',
                  left_on = ['dropoff_longitude','dropoff_latitude'],
                  right_on = ['dropoff_longitude','dropoff_latitude'])

In [ ]:
# Find null locations id
trips_with_zones.loc[(trips_with_zones.dropoff_location_id.isnull()) | (trips_with_zones.pickup_location_id.isnull())]

Out of City ID for trips when data was not limited to the boundary box (optional code above).

In [ ]:
# Out if City ID
OoC_ID = 0

Generate code for trips in january only (smaller data size for faster model training)

In [ ]:
# Select the desired columns
january = trips_with_zones[['medallion', 'pickup_week_day', 'pickup_hour', 'pickup_month', 'pickup_location_id', 'dropoff_location_id']].copy()

In [ ]:
# From NaN to 0 pickup
january.loc[january.pickup_location_id.isnull(), 'pickup_location_id'] = 0.0

# From NaN to 0 dropoff
january.loc[january.dropoff_location_id.isnull(), 'dropoff_location_id'] = 0.0

In [ ]:
# Save the dataset
january.to_csv('./zones_results/trips_with_zones_1.csv', index=False)

# ORIGINAL CODE:
Below is the original unedited code of the referenced work for the preprocessing with the original notes and conclusions.

# Pre-Processing Pipeline

## Import the dataset into the database

In [ ]:
# Insert datasets to the db
# Insert datasets from january to ....
months = 2
j, chunksize = 1, 100000
for month in range(1,months):
    fp = "./trip_data/trip_data_{0}.csv".format(month)
    for df in pd.read_csv(fp, chunksize=chunksize, iterator=True):
        df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns})
        df['pickup_hour'] = [x[11:16] for x in df['pickup_datetime']]
        df['dropoff_hour'] = [x[11:16] for x in df['dropoff_datetime']]
        df['pickup_date'] = [x[0:11] for x in df['pickup_datetime']]
        df['dropoff_date'] = [x[0:11] for x in df['dropoff_datetime']]
        df.drop(['vendor_id', 'rate_code', 'store_and_fwd_flag', 'trip_time_in_secs'], axis=1, inplace=True)
        df.index += j
        df.to_sql('trip_record', nyc_database, if_exists='append')
        j = df.index[-1] + 1
del df

In [ ]:
# If we have some problem, just drop the table

# DROP TABLE
#pd.read_sql_query('DROP TABLE IF EXISTS trip_record', nyc_database)

In [ ]:
# Total number of records
number = pd.read_sql_query('SELECT count(*) as number FROM trip_record', nyc_database)
print("Total number of records: ", number['number'][0])

## Find invalid records

In [ ]:
# Helper values to define validity

# Find the outliers
min_lat= 40.5774
max_lat= 40.9176
min_long= -74.15
max_long= -73.7004

# valid area (bounding box around the states)
min_lat= 38.8472
max_lat= 45.0153
min_long= -80.5243
max_long= -71.7517


# Dates in years in the past or in the future
upper = '2013-12-31 23:59:59'
lower = '2013-01-01 00:00:00'

# Logic operators
OR = ' OR '
AND = ' AND '
NOT = 'NOT '

In [ ]:
# Records with null coordinates
c1 = ( '((pickup_latitude IS NULL OR dropoff_latitude IS NULL) OR ' + 
         '(pickup_longitude IS NULL OR dropoff_longitude IS NULL))')

# Records with Coordinates out of range
c3 = ('((dropoff_longitude NOT BETWEEN :min_long AND :max_long ) OR '+
         '(dropoff_latitude NOT BETWEEN :min_lat AND :max_lat)) OR '+
      '((pickup_longitude NOT BETWEEN  :min_long AND :max_long ) OR '+
         '(pickup_latitude NOT BETWEEN  :min_lat AND :max_lat))')

# Records with pickup coordinates equal to dropoff
c2 = ('((pickup_latitude = dropoff_latitude) AND ' +
         '(pickup_longitude = dropoff_longitude ))')

# Records with distance 0
c4 = '(trip_distance <= 0.1)'

# Records with pickup time equal or greater than dropoff time
c5_7_9_10 = '(pickup_datetime >= dropoff_datetime)'

# Records with dates in years in the past or in the future
c6 = ('((pickup_datetime > :upper OR pickup_datetime < :lower ) OR (dropoff_datetime > :upper OR dropoff_datetime < :lower ))')

# Records with zero or null passengers
c8 = '(passenger_count < 1 OR passenger_count IS null OR passenger_count > 6)'

# null datetimes
c9 = '(pickup_datetime IS NULL OR dropoff_datetime IS NULL)'

not_where_clause = '(' + c1 + OR + c3 + OR + c2 + OR + c5_7_9_10 + OR + c6+ OR + c8 + OR + c9 +')'
# count invalid rows
query = ('SELECT count(*) as number FROM trip_record WHERE '+ not_where_clause) # c1 + OR + c3 + OR + c2 + OR + c5_7_9_10 + OR + c6  
res = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})
invalid = res['number'][0]
print('Number of invalid rows: ', invalid)

## Find valid records

In [ ]:
# Records with not null coordinates
c1 = ( '((pickup_latitude IS NOT NULL AND dropoff_latitude IS NOT NULL) AND ' + 
         '(pickup_longitude IS NOT NULL AND dropoff_longitude IS NOT NULL))')

# Records with coordinates in the range
c3 = ('((dropoff_longitude BETWEEN :min_long AND :max_long ) AND '+
         '(dropoff_latitude BETWEEN :min_lat AND :max_lat) AND '+
      '(pickup_longitude BETWEEN :min_long AND :max_long ) AND '+
         '(pickup_latitude BETWEEN :min_lat AND :max_lat))')

# Records with pickup coordinates not equal to dropoff
c2 = ('((pickup_latitude != dropoff_latitude) OR ' +
         '(pickup_longitude != dropoff_longitude ))')

# Records with different coordinates but distance greater 0
c4 = '(trip_distance > 0.1)'

# Records with pickup time smaller than dropoff time
c5_7_9_10 = '(pickup_datetime < dropoff_datetime)'

# Records with dates in the right range
c6 = ('((pickup_datetime <= :upper AND pickup_datetime >= :lower ) AND (dropoff_datetime <= :upper AND dropoff_datetime >= :lower ))')

# Not Zero passengers and passenger count in range
c8 = '((passenger_count BETWEEN 1 AND 6) AND passenger_count IS NOT null )'

# not null datetimes
c9 = '(pickup_datetime IS NOT NULL AND dropoff_datetime IS NOT NULL)'

where_clause = '(' + c1 + AND + c3 + AND + c2 + AND + c5_7_9_10 + AND + c6 + AND + c8 + AND + c9 + ')'


In [ ]:

query = ('SELECT count(*) as number FROM trip_record WHERE '+where_clause ) # c1 + AND + c3 + AND + c2 + AND + c5_7_9_10 + AND + c6
res = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})
valid = res['number'][0]
print('Number of valid rows: ', valid)

# to check if number is correct
#print( 'Tot: ', valid + invalid)
#print( 'Error: ', 14776615 - (valid + invalid))

### Retrieve valid records

In [ ]:
query = ('SELECT *  FROM trip_record WHERE '+ where_clause + ' LIMIT 1000') #+ ' LIMIT 1000000'
valid_records = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})

In [ ]:
valid_records.shape 

Read the records in the valid area chunk by chunk to avoid memory overflow

In [ ]:
import pandas.io.sql as psql

chunk_size = 5000000
offset = 0
dfs = []
header = True
while True:
  print(offset)
  sql = "SELECT * FROM trip_record WHERE " + where_clause + " ORDER BY pickup_longitude LIMIT %d offset %d " % (chunk_size,offset) 
  df = pd.read_sql_query( sql, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})
  print('Query done')
  df.to_csv('./ma_results/january.csv', header=header, mode='a')

  

  header = False
  offset += chunk_size
  if len(df) < chunk_size:
    break
  del df
  gc.collect()


In [ ]:
  # Empty space in the memory
  del df
  gc.collect()

In [ ]:
# Save the partial result
df = pd.read_csv('./ma_results/january.csv')

In [ ]:
df.head()

### Google Drive version

In [ ]:
!pip install pyshp
!pip install haversine
!pip install geopandas
!pip install pygeos

!sudo apt install libspatialindex-dev

!pip install Rtree

# Imports
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
from tqdm import tqdm
import time
from datetime import datetime
import timeit

# libraries to visualize data
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline


# compute additional features
from haversine import haversine
from pandas import DataFrame
from numba import njit
from math import radians, cos, sin, asin, sqrt

In [ ]:
dtypes = {'Unnamed: 0':'str',	'index':'str',	'medallion':'str',	'hack_license':'str',	'pickup_datetime':'str',	'dropoff_datetime':'str',	'passenger_count':'str',	'trip_distance':'str',
          'pickup_longitude':'str',	'pickup_latitude':'str',	'dropoff_longitude':'str',	'dropoff_latitude':'str',	'pickup_hour':'str',	'dropoff_hour':'str',	'pickup_date':'str',	'dropoff_date':'str'}
usecols=['medallion',	'hack_license',	'pickup_datetime',	'dropoff_datetime',	'passenger_count',	'trip_distance',
          'pickup_longitude',	'pickup_latitude',	'dropoff_longitude',	'dropoff_latitude',	'pickup_hour',	'dropoff_hour',	'pickup_date',	'dropoff_date']

path = './ma_results'
valid_records = pd.read_csv(path + '/january.csv', usecols=usecols)

In [ ]:
valid_records.head()

## Elaborate the data
Computing extra features to clean the dataset

Helper functions

In [ ]:
from haversine import haversine
from datetime import datetime
from pandas import DataFrame
import math
import numpy as np

In [ ]:
# https://www.kaggle.com/kartikkannapur/nyc-taxi-trips-exploratory-data-analysis
# compute haversine distance of a trip
# *The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.*

**Moving to numba to improve performances**

https://stackoverflow.com/questions/52673285/performance-of-pandas-apply-vs-np-vectorize-to-create-new-column-from-existing-c

In [ ]:
from numba import njit

# fix problematic times and conver time into hours
@njit
def refine_time(time_difference):
  res = np.empty(time_difference.shape)
  for i in range(len(time_difference)):
    duration_hours = time_difference[i]/3600.0
    if duration_hours < 0.005:
      trip_time = 0.005
    else:
      trip_time = round(duration_hours, 3)
    res[i] = trip_time
  return res



---
Duration of the trips


In [ ]:
valid_records["pickup_datetime"] = pd.to_datetime(valid_records["pickup_datetime"])
valid_records["dropoff_datetime"] = pd.to_datetime(valid_records["dropoff_datetime"])
valid_records['trip_time'] = (valid_records['dropoff_datetime'] - valid_records['pickup_datetime']).dt.total_seconds()
#%timeit valid_records['seconds'] = refine_time(valid_records['seconds'].values)
#print(refine_time(valid_records['seconds'].values))
valid_records['trip_time'] = refine_time(valid_records['trip_time'].values)
valid_records['trip_time']

**%timeit in front a that function cause problem, the values returned become all 0.005**



---
Haversine Distance of the trips

https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas


In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine_np( lat1, lon1, lat2, lon2):
    # earth radius
    R = 6371.0
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = R * c
    return np.round(km, 3)

In [ ]:
valid_records['haversine_distance'] = haversine_np(valid_records["pickup_latitude"], valid_records["pickup_longitude"], 
                     valid_records["dropoff_latitude"], valid_records["dropoff_longitude"])

In [ ]:
valid_records['haversine_distance']



---
Speed of the trips


In [ ]:
valid_records['speed'] = round(valid_records['haversine_distance']/valid_records['trip_time'], 1)
valid_records['speed']

## Statistics on valid records

### Number of passengers

In [ ]:
mean_p = valid_records["passenger_count"].mean()
max_p = valid_records["passenger_count"].max()
min_p = valid_records["passenger_count"].min()

print("The mean number of passengers is: ", mean_p)
print("The max number of passengers is: ", max_p)
print("The min number of passengers is: ", min_p)


### Compute mean, min and max ***trip distance***


In [ ]:
mean_d = valid_records["haversine_distance"].mean()
max_d = valid_records["haversine_distance"].max()
min_d = valid_records["haversine_distance"].min()

print("The mean distance is: ", mean_d)
print("The max distance is: ", max_d)
print("The min distance is: ", min_d)
print("The distances are represented in Km unit")

### Compute mean, min and max ***trip time***


In [ ]:
mean_t = valid_records["trip_time"].mean()
max_t = valid_records["trip_time"].max()
min_t = valid_records["trip_time"].min()

print("The mean trip_time is: ", mean_t)
print("The max trip_time is: ", max_t)
print("The min trip_time is: ", min_t)
print("The trip_times are represented in hours")

### Compute mean, min and max ***speed***

In [ ]:
mean_speed = valid_records["speed"].mean()
max_speed = valid_records["speed"].max()
min_speed = valid_records["speed"].min()

print("The mean speed is: ", mean_speed)
print("The max speed is: ", max_speed)
print("The min speed is: ", min_speed)

In [ ]:
# because haversine distance is the semicircle on  the sphere which connect 2 points, in reality the distance between A and B is greater, 
# so the real speed is higher than the one computed with haversine

print("records: ", len(valid_records.medallion))
valid_records.haversine_distance.hist(bins=30, figsize=(10,5), color = 'blue')
plt.xlabel('Haversine ditance')
plt.title('Histogram');
plt.show() 

valid_records.trip_time[valid_records.trip_time<8].hist(bins=30, figsize=(10,5), color = 'red')
plt.xlabel('Trip time')
plt.title('Histogram');
plt.show() 

valid_records.speed[valid_records.speed<100].hist(bins=20, figsize=(8,5), color = 'green')
plt.xlabel('Speed')
plt.title('Histogram');

## Statistics to find correct speed, time and distance ranges

Time
distance
speed

https://medium.com/datadriveninvestor/finding-outliers-in-dataset-using-python-efc3fce6ce32


step 1:
* Arrange the data in increasing order
* Calculate first(q1) and third quartile(q3)
* Find interquartile range (q3-q1)
* Find lower bound q1*1.5
* Find upper bound q3*1.5
* Anything that lies outside of lower and upper bound is an outlier


In [ ]:
from scipy import stats
# Haversine distance
distances = np.array(valid_records.sort_values(by=['haversine_distance']).haversine_distance)
q1, q3= np.percentile(distances,[25,75])
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr) 
print("IQR")
print("Lower bound: {}, Upper bound: {}".format(lower_bound,upper_bound))

Try with the Z score

In [ ]:
#distances = np.array(valid_records.sort_values(by=['haversine_distance']).haversine_distance)
z = np.abs(stats.zscore(distances))
threshold = 3
idx = np.where(z > threshold)
lower_bound = distances[idx].min()
upper_bound = distances[idx].max()
print("Z-score")
print("Upper bound from: {}, to: {}".format(lower_bound,upper_bound))

idx = np.where(z < -threshold)
if(len(idx[0]) >0):
  distances[idx].max()

Trip time

In [ ]:
times = np.array(valid_records.sort_values(by=['trip_time']).trip_time)
q1, q3= np.percentile(times,[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 
print("IQR")
print("Lower bound: {}, Upper bound: {}".format(lower_bound,upper_bound))
print("")
# Z score
z = np.abs(stats.zscore(times))
threshold = 3
idx = np.where(z > threshold)
lower_bound = times[idx].min()
upper_bound = times[idx].max()
print("Z-score")
print("Upper bound from: {}, to: {}".format(lower_bound,upper_bound))

idx = np.where(z < -threshold)
if(len(idx[0]) >0):
  times[idx].max()

Looking at the trips with triptime greater than 22 hours they seems weird. The number of these records is very low so we can remove them

In [ ]:
valid_records.loc[valid_records['trip_time']>22.0]

Trips longer than 4 hourse

In [ ]:
valid_records.loc[valid_records['trip_time']>4.0].sort_values(by=['trip_time'], ascending=False).shape

Speed

In [ ]:
speeds = np.array(valid_records.sort_values(by=['speed']).speed)
q1, q3= np.percentile(speeds,[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 
print("IQR")
print("Lower bound: {}, Upper bound: {}".format(lower_bound,upper_bound))
print("")
# Z score
z = np.abs(stats.zscore(speeds))
threshold = 3
idx = np.where(z > threshold)
lower_bound = speeds[idx].min()
upper_bound = speeds[idx].max()
print("Z-score")
print("Upper bound from: {}, to: {}".format(lower_bound,upper_bound))

idx = np.where(z < -threshold)
if(len(idx[0]) >0):
  speeds[idx].max()

### Conclusions


Is it not possible to relay just on this statistical measures, considering the context i will use these treesholds:

**Haversine Distance:**
*   Lower bound: 0.1 km
*   Upper bound: 70 km

---
**Trip time:**
With our location granularity a movement of less than 6 minutes probably would not be enough to change zone

4 hours, looking at the data seems the extreme case
*   Lower bound: 0.1 h
*   Upper bound: 4.0 h

---
From: https://www.kaggle.com/headsortails/nyc-taxi-eda-update-the-fast-the-curious#direct-distance-of-the-trip

*Well, after removing the most extreme values this looks way better than I would have expected. An average speed of around 15 km/h sounds probably reasonable for NYC. Everything above 50 km/h certainly requires magical cars (or highway travel). Also keep in mind that this refers to the direct distance and that the real velocity would have been always higher.*

*In a similar way as the average duration per day and hour we can also investigate the average speed for these time bins:*

https://www.latimes.com/nation/la-na-new-york-traffic-manhattan-20180124-story.html

**Speed:**
*   Lower bound: 5 km/h
*   Upper bound: 60 km/h


In [ ]:
# Treesholds
haversine_distance_t = (0.1, 70)
trip_time_t = (0.1, 4.0)
speed_t = (5, 60)

# define more restrictive thresholds for the intermediate trips
intermediate_distance_t = (0.5, 20)
intermediate_trip_time_t = (0.1, 1.0)
intermediate_speed_t = (5, 40)

## Remove outliers before intermediate trips

Remove trips with unreal speed

In [ ]:
# speed
valid_records = valid_records.loc[(valid_records.speed >= speed_t[0]) & 
                                          (valid_records.speed <= speed_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["speed"])), np.sort(valid_records["speed"].values), color = 'green')
plt.xlabel('index')
plt.ylabel('speed in km/h')
plt.show()

Remove trips with unreal haversine distance

In [ ]:
# haversine_distance
valid_records = valid_records.loc[(valid_records.haversine_distance >= haversine_distance_t[0]) & 
                                          (valid_records.haversine_distance <= haversine_distance_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["haversine_distance"])), np.sort(valid_records["haversine_distance"]), color = 'blue')
plt.xlabel('index')
plt.ylabel('haversine_distance in km')
plt.show()

Remove trips with unreal trip time

In [ ]:
# trip_time
valid_records = valid_records.loc[(valid_records.trip_time >= trip_time_t[0]) & 
                                          (valid_records.trip_time <= trip_time_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(valid_records["trip_time"])), np.sort(valid_records["trip_time"]), color = 'red')
plt.xlabel('index')
plt.ylabel('trip_time in km')
plt.show()

Final amount of valid records

In [ ]:
valid_records.shape

## Add intermediate trips

In [ ]:
# Treesholds
haversine_distance_t = (0.1, 70)
trip_time_t = (0.1, 4.0)
speed_t = (5, 60)

# define more restrictive thresholds for the intermediate trips
intermediate_distance_t = (0.5, 20)
intermediate_trip_time_t = (0.1, 1.0)
intermediate_speed_t = (5, 40)

In [ ]:
# Retrieve the valid trips
query = ('SELECT medallion, hack_license, pickup_datetime, dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude '+
         'FROM trip_record WHERE '+ where_clause + ' ORDER BY medallion, pickup_datetime') #LIMIT 100000
t1 = datetime.now()
valid_records = pd.read_sql_query( query, nyc_database, params={"min_long":min_long, "max_long":max_long, "min_lat":min_lat, "max_lat":max_lat, "upper":upper, "lower":lower})
t2 = datetime.now()
delta = t2 - t1
print("time for query: {}".format(delta.seconds))

**To iterate across pairs of rows we need to make a copy of our dataframe and shif it by -1, so the line 1 will be concatenate with the line 2 and so on.**

In [ ]:
# Convert the datetimes from strings to datetimes
valid_records["pickup_datetime"] = pd.to_datetime(valid_records["pickup_datetime"])
valid_records["dropoff_datetime"] = pd.to_datetime(valid_records["dropoff_datetime"])

In [ ]:
# sort the records
valid_records.sort_values(by=['medallion', 'pickup_datetime'], inplace=True)

In [ ]:
# Rename the columns for the dataframe copy
print("Renaming the new columns...")
new_columns_name = np.apply_along_axis(lambda column: column+'_b', 0, np.array(valid_records.columns))

# Copy the dataframe and use the new columns names (takes some time..)
print("Copying the dataframe...")
#valid_records_copy = pd.DataFrame(data=valid_records.values, columns=new_columns_name)
valid_records_copy  = valid_records.copy()
valid_records_copy.columns = new_columns_name

# Shift the copy dataframe
print("Shifting the copied dataframe...")
valid_records_copy = valid_records_copy.shift(-1)

# concatenate the dataframes 
print("Concatenating the two dataframes...")
concatenated_dataframes = pd.concat([valid_records, valid_records_copy], axis=1, sort=False).dropna()

In [ ]:
concatenated_dataframes.dtypes

To generate the intermediate trips we need a function to apply on each row.

This function will make some checks and then return the new computed row.

If an intermediate trip is not possibile, the function returns a None row.


In [ ]:
# Min time allowed between two trips and min intermediate trip length
min_time = intermediate_trip_time_t[0]
# Max time allowed between two trips and max intermediate trip length
max_time = intermediate_trip_time_t[1]

# Min distance allowed between two trips and min intermediate trip distance
min_distance = intermediate_distance_t[0]
# Max distance allowed between two trips and max intermediate trip distance
max_distance = intermediate_distance_t[1]

# Min speed allowed between two trips and min intermediate trip speed
min_speed = intermediate_speed_t[0]
# Max speed allowed between two trips and max intermediate trip speed
max_speed = intermediate_speed_t[1]

# we need to reduce the number of rows first
concatenated_dataframes['intermediate_time'] = (concatenated_dataframes['pickup_datetime_b'] - 
                                                concatenated_dataframes['dropoff_datetime']).dt.total_seconds()
concatenated_dataframes['intermediate_time'] = refine_time(concatenated_dataframes['intermediate_time'].values)

concatenated_dataframes['intermediate_distance'] = haversine_np(concatenated_dataframes["pickup_latitude_b"].values, concatenated_dataframes["pickup_longitude_b"].values, 
                     concatenated_dataframes["dropoff_latitude"].values, concatenated_dataframes["dropoff_longitude"].values)
concatenated_dataframes['intermediate_speed'] = round(concatenated_dataframes['intermediate_distance']/concatenated_dataframes['intermediate_time'], 1)

candidates = concatenated_dataframes.loc[((concatenated_dataframes['medallion'] == concatenated_dataframes['medallion_b']) & (concatenated_dataframes['hack_license'] == concatenated_dataframes['hack_license_b'])) &
                                         ((concatenated_dataframes['intermediate_time'] <= max_time) & (concatenated_dataframes['intermediate_time'] >= min_time)) & 
                                         ((concatenated_dataframes['intermediate_distance'] <= max_distance) & (concatenated_dataframes['intermediate_distance'] >= min_distance)) &
                                         ((concatenated_dataframes['intermediate_speed'] <= max_speed) & (concatenated_dataframes['intermediate_speed'] >= min_speed))]
candidates.shape
                              

In [ ]:
# Change the columns names
intermediate_trips = candidates[['medallion', 'hack_license', 'dropoff_datetime', 'pickup_datetime_b', 'dropoff_longitude', 'dropoff_latitude',
                                 'pickup_longitude_b','pickup_latitude_b', 'intermediate_distance', 'intermediate_time','intermediate_speed']]

intermediate_trips = intermediate_trips.rename(columns={'medallion':'medallion', 'hack_license':'hack_license', 'dropoff_datetime':'pickup_datetime', 'pickup_datetime_b':'dropoff_datetime', 
                                   'pickup_longitude_b':'dropoff_longitude',
                                   'pickup_latitude_b':'dropoff_latitude', 
                                    'dropoff_longitude':'pickup_longitude',
                                   'dropoff_latitude':'pickup_latitude', 
                                   'intermediate_distance': 'haversine_distance',
                                   'intermediate_time':'trip_time', 'intermediate_speed':'speed'})

In [ ]:
intermediate_trips

In [ ]:
# compute extra features now, before now it would have just added complexity to the big operations we made to compute the intermediate trips
valid_records['trip_time'] = (valid_records['dropoff_datetime'] - valid_records['pickup_datetime']).dt.total_seconds()
valid_records['trip_time'] = refine_time(valid_records['trip_time'].values)
valid_records['haversine_distance'] = haversine_np(valid_records["pickup_latitude"], valid_records["pickup_longitude"], 
                     valid_records["dropoff_latitude"], valid_records["dropoff_longitude"])
valid_records['speed'] = round(valid_records['haversine_distance']/valid_records['trip_time'], 1)

# Union of the original valid trips and the new intermediate trips
all_valid_records = pd.concat([valid_records, intermediate_trips], axis=0)

# Sort the records by medallion and pickup_time
all_valid_records = all_valid_records.sort_values(by=['medallion', 'pickup_datetime'])

In [ ]:
all_valid_records.head(20)

In [ ]:
all_valid_records.shape

## Statistics on full dataset
*From this analysis we can see that there are few outlier considering the values of distance, time and speed*

*We should remove them*

### First and Last date


In [ ]:
# pickup
max_pickup_date = all_valid_records["pickup_datetime"].max()
min_pickup_date= all_valid_records["pickup_datetime"].min()


# dropoff
max_dropoff_date = all_valid_records["dropoff_datetime"].max()
min_dropoff_date= all_valid_records["dropoff_datetime"].min()



print("Pickup: ")
print("Min Date: ", min_pickup_date)
print("Max Date: ", max_pickup_date)


print(" ")

print("Dropoff: ")
print("Min Date: ", min_dropoff_date)
print("Max Date: ", max_dropoff_date)

### Min and Max **Longitude** and **Latitude** for Pickup and Dropoff


In [ ]:

# pickup
max_pickup_longitude = all_valid_records["pickup_longitude"].max()
min_pickup_longitude= all_valid_records["pickup_longitude"].min()
max_pickup_latitude = all_valid_records["pickup_latitude"].max()
min_pickup_latitude= all_valid_records["pickup_latitude"].min()

# dropoff
max_dropoff_latitude = all_valid_records["dropoff_latitude"].max()
min_dropoff_latitude= all_valid_records["dropoff_latitude"].min()
max_dropoff_longitude = all_valid_records["dropoff_longitude"].max()
min_dropoff_longitude= all_valid_records["dropoff_longitude"].min()


print("Pickup: ")
print("Min Latitude: ", min_pickup_latitude)
print("Min Longitude: ", min_pickup_longitude)
print("Max Latitude: ", max_pickup_latitude)
print("Max Longitude: ", max_pickup_longitude)

print(" ")

print("Dropoff: ")
print("Min Latitude: ", min_dropoff_latitude)
print("Min Longitude: ", min_dropoff_longitude)
print("Max Latitude: ", max_dropoff_latitude)
print("Max Longitude: ", max_dropoff_longitude)

### Most common medallion

In [ ]:
all_valid_records['medallion'].value_counts()

### Most common Medallion - Hack_license combination

In [ ]:
all_valid_records.groupby(['medallion','hack_license']).size().sort_values(ascending=False)
#counts.index[0] 

### Compute mean, min and max ***trip distance***


In [ ]:
mean_d = all_valid_records["haversine_distance"].mean()
max_d = all_valid_records["haversine_distance"].max()
min_d = all_valid_records["haversine_distance"].min()

print("The mean distance is: ", mean_d)
print("The max distance is: ", max_d)
print("The min distance is: ", min_d)
print("The distances are represented in Km unit")

In [ ]:
# plot values to spot outliers
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["haversine_distance"])), np.sort(all_valid_records["haversine_distance"]), color = 'red')
plt.xlabel('index')
plt.ylabel('haversine_distance in km')
plt.show()

### Compute mean, min and max ***trip time***


In [ ]:
mean_t = all_valid_records["trip_time"].mean()
max_t = all_valid_records["trip_time"].max()
min_t = all_valid_records["trip_time"].min()

print("The mean trip_time is: ", mean_t)
print("The max trip_time is: ", max_t)
print("The min trip_time is: ", min_t)
print("The trip_times are represented in hours")

In [ ]:
# Row with the max time
index = all_valid_records["trip_time"].argmax()
all_valid_records.iloc[index]

In [ ]:
# plot values to spot outliers
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["trip_time"])), np.sort(all_valid_records["trip_time"]),color = 'blue')
plt.xlabel('index')
plt.ylabel('trip_time in hours')
plt.show()

### Compute mean, min and max ***speed***

In [ ]:
mean_speed = all_valid_records["speed"].mean()
max_speed = all_valid_records["speed"].max()
min_speed = all_valid_records["speed"].min()

print("The mean speed is: ", mean_speed)
print("The max speed is: ", max_speed)
print("The min speed is: ", min_speed)

In [ ]:
# plot values to spot outliers
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["speed"])), np.sort(all_valid_records["speed"]), color = 'green')
plt.xlabel('index')
plt.ylabel('speed in km/h')
plt.show()

### Worth notice

From this analysis we can see that there are few outlier considering the values of distance, time and speed

We should remove them

## Remove outliers

Remove trips with unreal speed

In [ ]:
# speed
all_valid_records = all_valid_records.loc[(all_valid_records.speed >= speed_t[0]) & 
                                          (all_valid_records.speed <= speed_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["speed"])), np.sort(all_valid_records["speed"].values), color = 'green')
plt.xlabel('index')
plt.ylabel('speed in km/h')
plt.show()

Remove trips with unreal haversine distance

In [ ]:
# haversine_distance
all_valid_records = all_valid_records.loc[(all_valid_records.haversine_distance >= haversine_distance_t[0]) & 
                                          (all_valid_records.haversine_distance <= haversine_distance_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["haversine_distance"])), np.sort(all_valid_records["haversine_distance"]), color = 'blue')
plt.xlabel('index')
plt.ylabel('haversine_distance in km')
plt.show()

Remove trips with unreal trip time

In [ ]:
# trip_time
all_valid_records = all_valid_records.loc[(all_valid_records.trip_time >= trip_time_t[0]) & 
                                          (all_valid_records.trip_time <= trip_time_t[1])]
plt.figure(figsize=(10,10))
plt.scatter(range(len(all_valid_records["trip_time"])), np.sort(all_valid_records["trip_time"]), color = 'red')
plt.xlabel('index')
plt.ylabel('trip_time in km')
plt.show()

In [ ]:
all_valid_records.shape

## Datetimes

We have datetimes which are useful for the analysis, but for our NN we need to refactor these attributes:

*  week_day
*  hour
*  month

In [ ]:
# already did it
all_valid_records["pickup_datetime"] = pd.to_datetime(all_valid_records["pickup_datetime"])
all_valid_records["dropoff_datetime"] = pd.to_datetime(all_valid_records["dropoff_datetime"])

In [ ]:
all_valid_records["pickup_week_day"] = all_valid_records.pickup_datetime.dt.dayofweek
all_valid_records["pickup_hour"] = all_valid_records.pickup_datetime.dt.hour
all_valid_records["pickup_month"] = all_valid_records.pickup_datetime.dt.month
all_valid_records["pickup_day"] = all_valid_records.pickup_datetime.dt.day

all_valid_records["dropoff_week_day"] = all_valid_records.dropoff_datetime.dt.dayofweek
all_valid_records["dropoff_hour"] = all_valid_records.dropoff_datetime.dt.hour
all_valid_records["dropoff_month"] = all_valid_records.dropoff_datetime.dt.month
all_valid_records["dropoff_day"] = all_valid_records.dropoff_datetime.dt.day

In [ ]:
all_valid_records.head()

In [ ]:
import gc 

del valid_records
del valid_records_copy
del intermediate_trips
del concatenated_dataframes
del candidates

gc.collect()

In [ ]:
all_valid_records.to_csv('./ma_results/january_before_zones.csv', index = False)

In [ ]:
all_valid_records = pd.read_csv('./ma_results/january_before_zones.csv')

### Compute time differences

In [ ]:
records = all_valid_records[['medallion', 'pickup_datetime', 'dropoff_datetime']].copy()

In [ ]:
records.pickup_datetime = pd.to_datetime(records.pickup_datetime)
records.dropoff_datetime = pd.to_datetime(records.dropoff_datetime)

In [ ]:
count = records.medallion.value_counts()
count

In [ ]:
medallions = count.loc[count>1000].index

In [ ]:
records = records.loc[records.medallion.isin(medallions)]

Datetimes to timestamps

In [ ]:
records['pickup_ts'] = records.pickup_datetime.values.astype(np.int64) // 10 ** 9

In [ ]:
records['dropoff_ts'] = records.dropoff_datetime.values.astype(np.int64) // 10 ** 9

In [ ]:
import datetime 
# Count the records for each user
# count = records.medallion.value_counts()

# Select the users with a number of records greater than the median
# records = records[records['medallion'].isin(count.index[count > 30])]

# List to collect differences
time_diff = []

#sf.sort_values(by=['medallion', 'pickup_ts'], inplace=True)

# For each user
for n, medallion in enumerate(medallions):

  sf = records.loc[records.medallion == medallion].copy()
  
  # Get the timestamps of the observations
  pickup_time = sf.pickup_ts.values

  # Get the timestamps of the observations
  dropoff_time = sf.dropoff_ts.values

  # Create a temporary array
  copy = np.zeros(pickup_time.shape[0], dtype=np.int32)

  # Copy the end timestamps shifted right by 1
  copy[1:] = pickup_time[:pickup_time.shape[0]-1]

  # compute the time difference
  diff = dropoff_time - copy
  diff[0] = 0

  # Cast from seconds to hours
  diff = diff /3600

  time_diff.extend(list(diff))


In [ ]:
# Save the numpy
np.save('./ma_results/NYC_time_diff.npy', np.array(time_diff))

In [ ]:
import matplotlib.pyplot as plt

n_bins = 50
fig, ax = plt.subplots(figsize=(8, 4))

# plot the cumulative histogram
n, bins, patches = ax.hist(time_diff[:1000], n_bins, density=True, histtype='stepfilled',
                           cumulative=True, label='Empirical',  color = "skyblue", ec="skyblue")


# tidy up the figure
ax.legend(loc='right')
ax.set_title('Cumulative step histograms')
ax.set_xlabel('Time interval')
ax.set_ylabel('Likelihood of occurrence')

plt.show()

In [ ]:
# we can't plot time differences of 4000h... we need to remove them just for have a clean plot

time_diff_c = np.array(time_diff)[np.array(time_diff) < 150]
n_bins = 10
fig, ax = plt.subplots(figsize=(8, 4))
# plot the cumulative histogram
n, bins, patches = ax.hist(time_diff_c, n_bins, density=True, histtype='stepfilled',
                           cumulative=False, label='Empirical',  color = "skyblue", ec="skyblue")


# tidy up the figure
ax.legend(loc='right')
ax.set_title('Cumulative step histograms')
ax.set_xlabel('Time interval')
ax.set_ylabel('Likelihood of occurrence')

plt.show()

In [ ]:
print("Number of places with a time difference over 80 hours: ", np.sum(np.array(time_diff) > 80))
print("Number of places with a time difference less than 80 hours: ", np.sum(np.array(time_diff) <= 80))
print("Number of places with a time difference less than 20 hours: ", np.sum(np.array(time_diff) <= 20))
print("Number of places with a time difference less than 2 hours: ", np.sum(np.array(time_diff) <= 2))

In [ ]:
diff_t = np.array(time_diff)
print("Mean time difference: ", np.mean(diff_t))
print("Max time difference:", np.max(diff_t))
print("Min time difference: ", np.min(diff_t))

## Join location ID

To reduce at the minimum size the dimension of the dataset to upload on Google BigQuery, we can limited the columns to:


* **pickup_longitude**
* **pickup_latitude**
* **dropoff_longitude**
* **dropoff_latitude**

Then joining with the zones we can simply add the ZoneID, 
We will be able to match it with our local dataset.

The valid rows which are in the valid area but now within the NYC boudaries will be mapped to a Zone which represent the outside of the city

The other ones that are in the NYC boudary will be used in bigquery

from BigQuery we will get back these attributes:

[ pickup_longitude, pickup_latitude, dropoff_longitude dropoff_latitude, zone_id, zone_name, borough ] 


in all_valid_records we have some problem of duplicated index (cause from the intermediate trips?)

we need to reindex

In [ ]:
all_valid_records.reset_index(drop=True, inplace=True)

Take the columns we are interested for

In [ ]:
locations = all_valid_records[["pickup_longitude", "pickup_latitude","dropoff_longitude", "dropoff_latitude"]].copy()
locations.head(100)

Save the location data

In [ ]:
# save the csv
locations.to_csv('./ma_results/locations.csv', index=False)

Remove coordinates which are outside of NYC

**We don't need this, coordinates not in the zones are simply discarded in the spatial join**

In [ ]:
locations_nyc = locations

Read the taxi zone dataset shapefile

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString, Polygon, asMultiPoint
import fiona.crs

fp = "./zones/geo_export_be42d9eb-5829-406d-a057-01bd027a191b.shp"

# Read the data
zones = gpd.read_file(fp)
zones.head()

Check the dataset crs type

In [ ]:
zones.crs

Drop eventual null rows

In [ ]:
zones = zones[np.invert(zones.isna())]
zones

Check for index duplicates

In [ ]:
locations_nyc = pd.read_csv('./ma_results/locations.csv')

In [ ]:
locations_nyc[locations_nyc.index.duplicated()]

### Pickup coordinates

Stack latitude and longitude of each row into a numpy array of N x 2 elements

Then generate a list of N Points from that pair of coordinates

In [ ]:
# stack latitude and longitude together 
pickup = np.vstack((locations_nyc["pickup_longitude"].values, locations_nyc["pickup_latitude"].values)).T 

# Convert each couple into a point object
t1 = datetime.now()
points = [Point(i) for i in pickup]
t2 = datetime.now()
delta = t2 - t1
print("time for conversion: {}".format(delta.seconds))

* Create a GeoSeries from the list of Points
* Change the crs to match with the one of zones
* Create a GeoDataframe from the GeoSeries
* Drop any invalid rows

In [ ]:
gs = GeoSeries(points)
gs.crs = fiona.crs.from_epsg(4326)
gd = gpd.GeoDataFrame(geometry=gs)
gd =  gd.loc[gd.is_valid]

In [ ]:
# Akternative method to change the crs
#gs.crs = {'init': 'epsg:4326', 'no_defs': True}
#gd = gpd.GeoDataFrame(geometry=gs)

In [ ]:
gd.head()

Add the original coordinates to the GeoDataframe, so we will have them in the result and we can use them to join back to the dataset

In [ ]:
gd['pickup_longitude'] = locations_nyc["pickup_longitude"].values
gd['pickup_latitude'] = locations_nyc["pickup_latitude"].values

In [ ]:
gd.head()

Check the DeoDataframe crs type

In [ ]:
gd.crs

Perform a spatial join between out points and the taxi zones

In [ ]:
join = gpd.sjoin(gd, zones, how="inner", op="within")
join.head()

Create a pandas dataframe with the non spatial data to join back with the original dataset

In [ ]:
# pickup_longitude | pickup_latitude | borough | location_i | zone
pickup_non_spacial_data = pd.DataFrame({'pickup_longitude': join['pickup_longitude'], 'pickup_latitude':join['pickup_latitude'] ,
                                 'pickup_borough': join['borough'], 'pickup_location_id': join['location_i'], 'pickup_zone': join['zone']})

In [ ]:
pickup_non_spacial_data.head()

### Dropoff

Same thing with the dropoff coordinates

In [ ]:
# stack latitude and longitude together 
dropoff = np.vstack((locations_nyc["dropoff_longitude"].values, locations_nyc["dropoff_latitude"].values)).T 

# Convert each couple into a point object
t1 = datetime.now()
points = [Point(i) for i in dropoff]
t2 = datetime.now()
delta = t2 - t1
print("time for conversion: {}".format(delta.seconds))

In [ ]:
gs = GeoSeries(points)
gs.crs = fiona.crs.from_epsg(4326)
gd = gpd.GeoDataFrame(geometry=gs)
gd =  gd.loc[gd.is_valid]

In [ ]:
gd['dropoff_longitude'] = locations_nyc["dropoff_longitude"].values
gd['dropoff_latitude'] = locations_nyc["dropoff_latitude"].values

In [ ]:
join = gpd.sjoin(gd, zones, how="inner", op="within")
join.head()

In [ ]:
# dropoff_longitude | dropoff_latitude | borough | location_i | zone
dropoff_non_spacial_data = pd.DataFrame({'dropoff_longitude': join['dropoff_longitude'], 'dropoff_latitude':join['dropoff_latitude'] ,
                                 'dropoff_borough': join['borough'], 'dropoff_location_id': join['location_i'], 'dropoff_zone': join['zone']})

### Check for index dulicates

There are coordinates which match with more than one zone and create duplicates

In [ ]:
all_valid_records = pd.read_csv('./ma_results/january_before_zones.csv')

In [ ]:
all_valid_records[all_valid_records.index.duplicated()]

In [ ]:
pickup_non_spacial_data[pickup_non_spacial_data.index.duplicated()]

In [ ]:
dropoff_non_spacial_data[dropoff_non_spacial_data.index.duplicated()]#.index[0]

If there is a duplicate, locate it by the index

In [ ]:
dropoff_non_spacial_data.loc[7971094]

**It is not a problem cause by my implementation, also BigQuery make this duplicate**

We can drop the duplicate and keep just the first result

In [ ]:
# Drop duplicates
dropoff_non_spacial_data = dropoff_non_spacial_data.loc[~dropoff_non_spacial_data.index.duplicated(keep='first')]

Check Most common combination of dropoff_longitude and dropoff_latitude

In [ ]:
# Most common combination of dropoff_longitude and dropoff_latitude
dropoff_non_spacial_data.groupby(['dropoff_longitude','dropoff_latitude']).size().sort_values(ascending=False)


Remove unecessary columns

In [ ]:
pickup_zones = pickup_non_spacial_data[ ['pickup_longitude', 'pickup_latitude', 'pickup_location_id']].copy()

In [ ]:
dropoff_zones = dropoff_non_spacial_data[ ['dropoff_longitude', 'dropoff_latitude', 'dropoff_location_id']].copy()

In [ ]:
# save the csv
pickup_zones.to_csv('./ma_results/pickup_zones_1.csv', index=False)
dropoff_zones.to_csv('./ma_results/dropoff_zones_1.csv', index=False)

In [ ]:
# read the csv
pickup_zones = pd.read_csv('./ma_results/pickup_zones_1.csv')
dropoff_zones = pd.read_csv('./ma_results/dropoff_zones_1.csv')

### Joining separately

In [ ]:
all_valid_records

In [ ]:
trips_with_zones = pd.merge(all_valid_records, pickup_zones,  how='left',
                  left_on=['pickup_longitude','pickup_latitude'],
                  right_on = ['pickup_longitude','pickup_latitude'])

In [ ]:
trips_with_zones.shape

In [ ]:
del trips_with_zones
gc.collect()

This merge returns 26M rows... this because there are duplicates in the pickup_zones table and they match with more than one row each.

we need to drop them

In [ ]:
pickup_zones = pickup_zones.drop_duplicates(subset=['pickup_longitude','pickup_latitude'])

In [ ]:
trips_with_zones = pd.merge(all_valid_records, pickup_zones,  how='left',
                  left_on=['pickup_longitude','pickup_latitude'],
                  right_on = ['pickup_longitude','pickup_latitude'])

In [ ]:
dropoff_zones = dropoff_zones.drop_duplicates(subset=['dropoff_longitude','dropoff_latitude'])

In [ ]:
trips_with_zones = pd.merge(trips_with_zones, dropoff_zones,  how='left',
                  left_on = ['dropoff_longitude','dropoff_latitude'],
                  right_on = ['dropoff_longitude','dropoff_latitude'])

In [ ]:
trips_with_zones

In [ ]:
trips_with_zones.loc[(trips_with_zones.dropoff_location_id.isnull()) | (trips_with_zones.pickup_location_id.isnull())]

## Last operations

Check the min location ID already used

In [ ]:
zones.location_i.min()

Set the Out of the City location ID to 0

In [ ]:
OoC_ID = 0

Extract the columns we will need for training

In [ ]:
trips_with_zones.columns

In [ ]:
january = trips_with_zones[['medallion', 'pickup_week_day', 'pickup_hour', 'pickup_day', 'pickup_month', 'dropoff_week_day', 'dropoff_hour', 'dropoff_day', 'dropoff_month', 'pickup_location_id', 'dropoff_location_id']]#.copy()

In [ ]:
january.loc[january.pickup_location_id.isnull(), 'pickup_location_id'] = 0.0

In [ ]:
january.loc[january.pickup_location_id == 0.0]

The same now with dropoff

In [ ]:
january.loc[january.dropoff_location_id.isnull(), 'dropoff_location_id'] = 0.0

In [ ]:
january.loc[january.dropoff_location_id == 0.0]

In [ ]:
january.loc[(january.dropoff_location_id == 0.0) & (january.pickup_location_id > 0.0)]

In [ ]:
# check for null values
january.isnull().sum()

In [ ]:
# Save the dataset
january.to_csv('./ma_results/trips_with_zones_final.csv', index=False)

In [ ]:
january.head(1000000).to_csv('./ma_results/trips_with_zones_10000000.csv', index=False)

In [ ]:
# how many rows?
df = pd.read_csv('./ma_results/trips_with_zones_10000000.csv')
df.head()

In [ ]:
df.info()

Save the pre-processed dataset

In [ ]:
df.to_csv('./ma_results/trips_1000000.csv', index=False)